# Compose: Training a model to generate music

In [3]:
import os
import pickle
import numpy
from music21 import note, chord

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model

from models.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network

## Set parameters

In [4]:
# run params
section = 'compose'
run_id = '0010'
music_name = 'cello'

run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])


store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('bach', music_name)

if not os.path.exists(run_folder):
    os.makedirs(run_folder)
    os.mkdir(os.path.join(run_folder, 'store'))
    os.mkdir(os.path.join(run_folder, 'output'))
    os.mkdir(os.path.join(run_folder, 'weights'))
    os.mkdir(os.path.join(run_folder, 'viz'))
    


mode = 'build' # 'load' # 

# data params
intervals = range(-2,3)
seq_len = 32

# model params
embed_size = 100
rnn_units = 128
use_attention = True

## Extract the notes

In [5]:
if mode == 'build':
    
    music_list, parser = get_music_list(data_folder)
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = parser.parse(file).chordify()
        

        for interval in intervals:

            score = original_score.transpose(interval)

            notes.extend(['START'] * seq_len)
            durations.extend([0]* seq_len)

            for element in score.flatten():
                
                if isinstance(element, note.Note):
                    if element.isRest:
                        notes.append(str(element.name))
                        durations.append(element.duration.quarterLength)
                    else:
                        notes.append(str(element.nameWithOctave))
                        durations.append(element.duration.quarterLength)

                if isinstance(element, chord.Chord):
                    notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                    durations.append(element.duration.quarterLength)

    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f) 

36 files in total
1 Parsing bach/cello/cs3-2all.mid
2 Parsing bach/cello/cs2-6gig.mid
3 Parsing bach/cello/cs6-3cou.mid
4 Parsing bach/cello/cs5-5gav.mid
5 Parsing bach/cello/cs2-4sar.mid
6 Parsing bach/cello/cs2-5men.mid
7 Parsing bach/cello/cs4-2all.mid
8 Parsing bach/cello/cs5-4sar.mid
9 Parsing bach/cello/cs5-6gig.mid
10 Parsing bach/cello/cs1-3cou.mid
11 Parsing bach/cello/cs1-1pre.mid
12 Parsing bach/cello/cs3-6gig.mid
13 Parsing bach/cello/cs3-4sar.mid
14 Parsing bach/cello/cs2-2all.mid
15 Parsing bach/cello/cs4-4sar.mid
16 Parsing bach/cello/cs6-1pre.mid
17 Parsing bach/cello/cs4-6gig.mid
18 Parsing bach/cello/cs5-2all.mid
19 Parsing bach/cello/cs6-2all.mid
20 Parsing bach/cello/cs3-3cou.mid
21 Parsing bach/cello/cs5-1pre.mid
22 Parsing bach/cello/cs1-2all.mid
23 Parsing bach/cello/cs2-1pre.mid
24 Parsing bach/cello/cs4-3cou.mid
25 Parsing bach/cello/cs6-4sar.mid
26 Parsing bach/cello/cs6-6gig.mid
27 Parsing bach/cello/cs4-1pre.mid
28 Parsing bach/cello/cs2-3cou.mid
29 Parsing 

## Create the lookup tables

In [6]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [7]:
print('\nnote_to_int')
note_to_int


note_to_int


{'A2': 0,
 'A2.A3': 1,
 'A2.A3.C#4.E-4': 2,
 'A2.A3.C#4.F#4': 3,
 'A2.A3.F#4': 4,
 'A2.B-2': 5,
 'A2.B-2.C3': 6,
 'A2.B-2.C3.D3': 7,
 'A2.B-3': 8,
 'A2.B2': 9,
 'A2.B3': 10,
 'A2.C#3': 11,
 'A2.C#3.D3.G3': 12,
 'A2.C#3.G#3': 13,
 'A2.C#3.G3': 14,
 'A2.C#4': 15,
 'A2.C#4.A4': 16,
 'A2.C#4.E4': 17,
 'A2.C3': 18,
 'A2.C3.A3.E4': 19,
 'A2.C4': 20,
 'A2.D3': 21,
 'A2.D3.C4': 22,
 'A2.D3.D4': 23,
 'A2.D4': 24,
 'A2.E-3': 25,
 'A2.E-3.C#4': 26,
 'A2.E-3.C4': 27,
 'A2.E3': 28,
 'A2.E3.A3': 29,
 'A2.E3.B-3': 30,
 'A2.E3.B3': 31,
 'A2.E3.B3.A4': 32,
 'A2.E3.C#4': 33,
 'A2.E3.C#4.A4': 34,
 'A2.E3.C#4.B4': 35,
 'A2.E3.C#4.E4': 36,
 'A2.E3.C#4.G#4': 37,
 'A2.E3.C#4.G4': 38,
 'A2.E3.C4': 39,
 'A2.E3.D4': 40,
 'A2.E3.D4.A4': 41,
 'A2.E3.E-4': 42,
 'A2.E3.E4': 43,
 'A2.E3.G3.A3.B-3': 44,
 'A2.F#3': 45,
 'A2.F#3.C#4': 46,
 'A2.F#3.C4': 47,
 'A2.F#3.D4': 48,
 'A2.F#3.D4.A4': 49,
 'A2.F#3.D4.E4': 50,
 'A2.F#3.D4.F#4': 51,
 'A2.F#3.E-4': 52,
 'A2.F#3.E-4.B4': 53,
 'A2.F#3.E4': 54,
 'A2.F#3.G3': 55,
 'A2.F

In [8]:
print('\nduration_to_int')
duration_to_int


duration_to_int


{0: 0,
 Fraction(1, 12): 1,
 Fraction(1, 6): 2,
 0.25: 3,
 Fraction(1, 3): 4,
 0.5: 5,
 Fraction(2, 3): 6,
 0.75: 7,
 1.0: 8,
 1.25: 9,
 Fraction(4, 3): 10,
 1.5: 11,
 1.75: 12,
 2.0: 13,
 2.5: 14,
 3.0: 15,
 4.0: 16}

## Prepare the sequences used by the Neural Network

In [9]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [10]:
print('pitch input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('pitch output')
print(network_output[0][0])
print('duration output')
print(network_output[1][0])

pitch input
[1388 1388 1388 1388 1388 1388 1388 1388 1388 1388 1388 1388 1388 1388
 1388 1388 1388 1388 1388 1388 1388 1388 1388 1388 1388 1388 1388 1388
 1388 1388 1388 1388]
duration input
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
pitch output
[0. 0. 0. ... 0. 0. 0.]
duration output
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [11]:
network_input[0]

array([[1388, 1388, 1388, ..., 1388, 1388, 1388],
       [1388, 1388, 1388, ..., 1388, 1388, 1112],
       [1388, 1388, 1388, ..., 1388, 1112, 1346],
       ...,
       [ 497,    0,  497, ...,   74, 1346, 1112],
       [   0,  497, 1045, ..., 1346, 1112,  497],
       [ 497, 1045,  497, ..., 1112,  497,  907]])

## Create the structure of the neural network

In [12]:
model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)
model.summary()

2024-01-19 07:45:41.359878: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 07:45:41.945792: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-01-19 07:45:41.945856: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78977 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:01:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    138900      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    1700        ['input_2[0][0]']                
                                                                                              

/opt/conda/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
!pip install pydot

In [ ]:
plot_model(model, to_file=os.path.join(run_folder ,'viz/model.png'), show_shapes = True, show_layer_names = True)

## Train the neural network

In [13]:
weights_folder = os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))

In [14]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=2000000, batch_size=64
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )



Epoch 1/2000000


2024-01-19 07:47:32.507628: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2024-01-19 07:47:32.702844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1786/1786 [==============================] - 28s 14ms/step - loss: 4.2169 - pitch_loss: 3.6157 - duration_loss: 0.6012 - val_loss: 4.1094 - val_pitch_loss: 3.3633 - val_duration_loss: 0.7461
Epoch 2/2000000
1786/1786 [==============================] - 24s 14ms/step - loss: 3.6613 - pitch_loss: 3.1984 - duration_loss: 0.4630 - val_loss: 3.9246 - val_pitch_loss: 3.2193 - val_duration_loss: 0.7052
Epoch 3/2000000
1786/1786 [==============================] - 25s 14ms/step - loss: 3.5159 - pitch_loss: 3.0923 - duration_loss: 0.4237 - val_loss: 3.9032 - val_pitch_loss: 3.2130 - val_duration_loss: 0.6902
Epoch 4/2000000
1786/1786 [==============================] - 25s 14ms/step - loss: 3.4152 - pitch_loss: 3.0223 - duration_loss: 0.3929 - val_loss: 3.9143 - val_pitch_loss: 3.2000 - val_duration_loss: 0.7143
Epoch 5/2000000
1786/1786 [==============================] - 25s 14ms/step - loss: 3.3219 - pitch_loss: 2.9621 - duration_loss: 0.3598 - val_loss: 3.9387 - val_pitch_loss: 3.2043 - val_dur

KeyboardInterrupt: 